In [1]:
import pymysql 
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from collections import defaultdict, Counter
import _pickle as pickle
import random
from scipy.stats import epps_singleton_2samp, wasserstein_distance, ks_2samp
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample
from lifelines import KaplanMeierFitter, CoxPHFitter

In [2]:
visit_probability = {}
for visit_id, prob in np.array(pd.read_csv('data/all_data_predictions_calibrated.csv', header=None)):
    visit_probability[int(visit_id)] = prob
    
len(visit_probability)

1573113

In [4]:
phenotype_visits_1y = pickle.load(open('data/phenotype_visits_1y_new.p', 'rb'))
followup_visits_1y = pickle.load(open('data/followup_visits_1y_new.p', 'rb'))
followup_tm_1y = pickle.load(open('data/followup_tm_1y_new.p', 'rb'))

In [5]:
demographics_variables = pickle.load(open('../vfinal_1/data/demographic_variables.p', 'rb'))
diag_variables = pickle.load(open('../vfinal_1/data/diag_variables.p', 'rb'))

In [6]:
all_visit_ids = set(followup_tm_1y.keys())

len(all_visit_ids)

525287

In [7]:
demographics_data = {}

file = open('../vfinal_1/data/all_visit_demographic_matrix.csv', 'r')

for line in tqdm(file):
    v_id = int(line.split(',')[0])
    if v_id not in all_visit_ids:
        continue
    dt = [int(i) for i in line.strip().split(',')[1:]]
    demographics_data[v_id]=dt

1573113it [00:02, 548137.32it/s]


In [8]:
file.close()

In [9]:
diag_data = {}

file = open('../vfinal_1/data/all_visit_diag_matrix.csv', 'r')

for line in tqdm(file):
    v_id = int(line.split(',')[0])
    if v_id not in all_visit_ids:
        continue
    dt = [int(i) for i in line.strip().split(',')[1:]]
    diag_data[v_id]=dt

1573113it [03:32, 7404.65it/s] 


In [10]:
file.close()

In [11]:
len(demographics_data), len(diag_data)

(525287, 525287)

In [12]:
previous_conditions = defaultdict(list)

file = open('data/previous_conditions_phe.csv', 'r')

for line in tqdm(file):
    if 'phecode' in line:
        continue
    v_id = int(line.split(',')[1])
    if v_id not in all_visit_ids:
        continue
    phe = float(line.split(',')[0])
    if phe not in previous_conditions[v_id]:
        previous_conditions[v_id].append(phe)
        
    

780740814it [22:37, 575119.35it/s]


In [13]:
file.close()

In [14]:
previous_conditions_phe_visit = defaultdict(list)

for visit_id in tqdm(previous_conditions):
    for phe in previous_conditions[visit_id]:
        previous_conditions_phe_visit[phe].append(visit_id)

100%|██████████| 444611/444611 [00:05<00:00, 80587.12it/s] 


In [15]:
cases_411_2 = []
for visit_id, diag_date, ed_date in phenotype_visits_1y['411.2']:
    if visit_id not in visit_probability:
        continue
    if (diag_date-ed_date).days <= 365:
        cases_411_2.append([visit_id, (diag_date-ed_date).days])
            
cases_411_2 = np.array(pd.DataFrame(cases_411_2))
non_cases_411_2 = list((set(followup_visits_1y[:,0])-set(cases_411_2[:,0]))-set(previous_conditions_phe_visit[411.2]))


cases_591 = []
for visit_id, diag_date, ed_date in phenotype_visits_1y['591']:
    if visit_id not in visit_probability:
        continue
    if (diag_date-ed_date).days <= 365:
        cases_591.append([visit_id, (diag_date-ed_date).days])
            
cases_591 = np.array(pd.DataFrame(cases_591))
non_cases_591 = list((set(followup_visits_1y[:,0])-set(cases_591[:,0]))-set(previous_conditions_phe_visit[591]))

cases_585_1 = []
for visit_id, diag_date, ed_date in phenotype_visits_1y['585.1']:
    if visit_id not in visit_probability:
        continue
    if (diag_date-ed_date).days <= 365:
        cases_585_1.append([visit_id, (diag_date-ed_date).days])
            
cases_585_1 = np.array(pd.DataFrame(cases_585_1))
non_cases_585_1 = list((set(followup_visits_1y[:,0])-set(cases_585_1[:,0]))-set(previous_conditions_phe_visit[585.1]))


cases_250_2 = []
for visit_id, diag_date, ed_date in phenotype_visits_1y['250.2']:
    if visit_id not in visit_probability:
        continue
    if (diag_date-ed_date).days <= 365:
        cases_250_2.append([visit_id, (diag_date-ed_date).days])
            
cases_250_2 = np.array(pd.DataFrame(cases_250_2))
non_cases_250_2 = list((set(followup_visits_1y[:,0])-set(cases_250_2[:,0]))-set(previous_conditions_phe_visit[250.2]))

cases_282_5 = []
for visit_id, diag_date, ed_date in phenotype_visits_1y['282.5']:
    if visit_id not in visit_probability:
        continue
    if (diag_date-ed_date).days <= 365:
        cases_282_5.append([visit_id, (diag_date-ed_date).days])
            
cases_282_5 = np.array(pd.DataFrame(cases_282_5))
non_cases_282_5 = list((set(followup_visits_1y[:,0])-set(cases_282_5[:,0]))-set(previous_conditions_phe_visit[250.2]))

In [16]:
len(cases_411_2), len(cases_591), len(cases_585_1), len(cases_250_2), len(cases_282_5), len(non_cases_411_2), len(non_cases_591), len(non_cases_585_1), len(non_cases_250_2), len(non_cases_282_5)

(6513, 12163, 17603, 9674, 4163, 488569, 419962, 450359, 420373, 421048)

In [17]:
condition_cts = Counter([phe for visit_id in set(followup_visits_1y[:,0]) for phe in set(previous_conditions[visit_id])])

In [18]:
pd.DataFrame([[phe, condition_cts[phe]] for phe in condition_cts], columns=['phe', 'cts']).sort_values(by='cts', ascending=False)[:30]


,phe,cts
4,401.10,226903
0,512.70,178181
3,272.10,156472
223,646.00,154584
84,512.80,149556
72,760.00,138735
212,1009.00,132551
5,530.11,126552
124,339.00,117382
20,508.00,115328


In [19]:
top_30 = list(pd.DataFrame([[phe, condition_cts[phe]] for phe in condition_cts], columns=['phe', 'cts']).sort_values(by='cts', ascending=False)[:30]['phe'])


In [20]:
previous_conditions_top_30 = {}

for visit_id in tqdm(set(followup_visits_1y[:,0])):
    d = []
    for phe in top_30:
        if phe in previous_conditions[visit_id]:
            d.append(1)
        else:
            d.append(0)
    previous_conditions_top_30[visit_id] = d
    

100%|██████████| 525287/525287 [00:10<00:00, 51643.81it/s]


In [24]:
coxph_model_data = []
for visit_id in set(cases_411_2[:,0]):
    time_to_diag = min(cases_411_2[:,1][cases_411_2[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_411_2:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/411_2_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()


ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [25]:
coxph_model_data = []
for visit_id in set(cases_411_2[:,0]):
    time_to_diag = min(cases_411_2[:,1][cases_411_2[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id][1:] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_411_2:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id][1:] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables[1:] + top_30 + ['covid_prob','phenotype', 'days']))
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/411_2_multi_30_cox_new.csv')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 489181 total observations, 488569 right-censored observations>
             duration col = 'days'
                event col = 'phenotype'
      baseline estimation = breslow
   number of observations = 489181
number of events observed = 612
   partial log-likelihood = -6698.85
         time fit was run = 2023-06-16 18:56:21 UTC

---
                                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                    
age_adol                              -1.46       0.23       0.74            -2.90            -0.01                 0.05                 0.99
age_adult                              0.39       1.48       0.23            -0.06             0.84                 0.94                 2.32
age_senior                             1.37       3.92       0.23             0.92             1.82                 2.50                 6.15
sex_1                                 -0.41       0.66       0.09            -0.58            -0.24                 0.56                 0.78
AMERICAN INDIAN OR ALASKA             -0.71       0.49       1.00            -2.67             1.26                 0.07                 3.51
ASIAN                                 -1.18       0.31       0.39            -1.93            -0.42                 0.15                 0.66
BLACK OR AFRICAN AMERICAN             -0.37       0.69       0.13            -0.62            -0.11                 0.54                 0.89
NAT.HAWAIIAN/OTH.PACIFIC               0.24       1.27       1.00            -1.73             2.20                 0.18                 9.05
WHITE                                 -0.45       0.63       0.10            -0.64            -0.27                 0.53                 0.77
HISPANIC OR LATINO OR SPANISH ORIGIN  -0.34       0.71       0.10            -0.54            -0.13                 0.58                 0.88
401.1                                 -0.37       0.69       0.11            -0.59            -0.15                 0.55                 0.86
512.7                                 -0.50       0.61       0.11            -0.71            -0.28                 0.49                 0.75
272.1                                 -0.03       0.97       0.12            -0.26             0.19                 0.77                 1.21
646.0                                 -0.21       0.81       0.16            -0.52             0.09                 0.60                 1.10
512.8                                 -0.17       0.85       0.12            -0.40             0.07                 0.67                 1.07
760.0                                  0.04       1.04       0.12            -0.19             0.27                 0.83                 1.31
1009.0                                -0.04       0.96       0.12            -0.28             0.20                 0.76                 1.22
530.11                                -0.13       0.88       0.12            -0.37             0.11                 0.69                 1.11
339.0                                 -0.06       0.94       0.13            -0.32             0.20                 0.72                 1.22
508.0                                  0.37       1.45       0.14             0.11             0.64                 1.11                 1.89
285.0                                 -0.17       0.84       0.13            -0.44             0.09                 0.65                 1.09
514.0                                 -0.35       0.70       0.14            -0.63            -0.07                 0.53                 0.93
783.0                                 -0.14       0.87       0.14            -0.42             0.13                 0.66                 1.14
591.0                                 -0.28       0.75       0.14            -0

In [27]:
coxph_model_data = []
for visit_id in set(cases_591[:,0]):
    time_to_diag = min(cases_591[:,1][cases_591[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_591:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(591.0, axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/591_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/fitters/coxph_fitter.py:1498: LinAlgWarning: Ill-conditioned matrix (rcond=1.04016e-16): result may not be accurate.
  inv_h_dot_g_T = spsolve(-h, g, assume_a="pos", check_finite=False)
/usr/local/lib/python3.9/site-packages/lifelines/fitters/coxph_fitter.py:1498: LinAlgWarning: Ill-conditioned matrix (rcond=3.59944e-17): result may not be accurate.
  inv_h_dot_g_T = spsolve(-h, g, assume_a="pos", check_finite=False)
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column NAT.HAWAIIAN/OTH.PACIFIC  have very low variance when conditioned on death event present or not. This may 

ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [28]:
coxph_model_data = []
for visit_id in set(cases_591[:,0]):
    time_to_diag = min(cases_591[:,1][cases_591[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id][:8] + demographics_data[visit_id][9:] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_591:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id][:8] + demographics_data[visit_id][9:] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables[:8] + demographics_variables[9:] + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(591.0, axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/591_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/fitters/coxph_fitter.py:1498: LinAlgWarning: Ill-conditioned matrix (rcond=1.04072e-16): result may not be accurate.
  inv_h_dot_g_T = spsolve(-h, g, assume_a="pos", check_finite=False)
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()


ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [30]:
coxph_model_data = []
for visit_id in set(cases_591[:,0]):
    time_to_diag = min(cases_591[:,1][cases_591[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id][1:8] + demographics_data[visit_id][9:] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_591:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id][1:8] + demographics_data[visit_id][9:] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables[1:8]+ demographics_variables[9:] + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(591.0, axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/591_multi_30_cox_new.csv')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 420988 total observations, 419962 right-censored observations>
             duration col = 'days'
                event col = 'phenotype'
      baseline estimation = breslow
   number of observations = 420988
number of events observed = 1026
   partial log-likelihood = -11641.85
         time fit was run = 2023-06-16 19:01:11 UTC

---
                                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                    
age_adol                              -1.02       0.36       0.25            -1.51            -0.54                 0.22                 0.58
age_adult                             -0.73       0.48       0.11            -0.94            -0.51                 0.39                 0.60
age_senior                            -0.30       0.74       0.12            -0.53            -0.07                 0.59                 0.93
sex_1                                  0.43       1.54       0.07             0.30             0.56                 1.35                 1.75
AMERICAN INDIAN OR ALASKA              0.16       1.18       0.50            -0.82             1.15                 0.44                 3.15
ASIAN                                  0.07       1.08       0.19            -0.30             0.45                 0.74                 1.57
BLACK OR AFRICAN AMERICAN              0.06       1.07       0.09            -0.12             0.25                 0.89                 1.28
WHITE                                 -0.07       0.94       0.07            -0.21             0.08                 0.81                 1.09
HISPANIC OR LATINO OR SPANISH ORIGIN   0.16       1.18       0.08             0.01             0.31                 1.01                 1.36
401.1                                 -0.21       0.81       0.09            -0.39            -0.03                 0.68                 0.97
512.7                                 -0.21       0.81       0.09            -0.38            -0.04                 0.69                 0.96
272.1                                 -0.18       0.84       0.10            -0.37             0.01                 0.69                 1.01
646.0                                 -0.18       0.83       0.13            -0.43             0.06                 0.65                 1.06
512.8                                 -0.19       0.83       0.09            -0.37            -0.01                 0.69                 0.99
760.0                                  0.02       1.02       0.10            -0.17             0.20                 0.84                 1.22
1009.0                                -0.01       0.99       0.09            -0.19             0.18                 0.82                 1.19
530.11                                -0.13       0.88       0.10            -0.32             0.06                 0.73                 1.06
339.0                                 -0.11       0.89       0.11            -0.32             0.09                 0.73                 1.10
508.0                                  0.03       1.03       0.12            -0.19             0.26                 0.82                 1.29
285.0                                  0.03       1.03       0.10            -0.17             0.22                 0.84                 1.25
514.0                                 -0.02       0.98       0.11            -0.23             0.20                 0.79                 1.22
783.0                                 -0.04       0.96       0.11            -0.25             0.17                 0.78                 1.18
250.2                                  0.23       1.26       0.10             0.04             0.42                 1.04                 1.52
563.0                                  0.36       1.43       0.10            

In [32]:
coxph_model_data = []
for visit_id in set(cases_585_1[:,0]):
    time_to_diag = min(cases_585_1[:,1][cases_585_1[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_585_1:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/585_1_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()


ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [33]:
coxph_model_data = []
for visit_id in set(cases_585_1[:,0]):
    time_to_diag = min(cases_585_1[:,1][cases_585_1[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id][1:] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_585_1:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id][1:] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables[1:] + top_30 + ['covid_prob','phenotype', 'days']))
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/585_1_multi_30_cox_new.csv')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 451912 total observations, 450359 right-censored observations>
             duration col = 'days'
                event col = 'phenotype'
      baseline estimation = breslow
   number of observations = 451912
number of events observed = 1553
   partial log-likelihood = -15930.88
         time fit was run = 2023-06-16 19:03:05 UTC

---
                                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                    
age_adol                               1.32       3.75       0.24             0.85             1.79                 2.35                 6.00
age_adult                              1.10       3.01       0.19             0.73             1.47                 2.08                 4.36
age_senior                             1.72       5.59       0.19             1.35             2.09                 3.85                 8.10
sex_1                                 -0.53       0.59       0.06            -0.64            -0.42                 0.53                 0.66
AMERICAN INDIAN OR ALASKA             -0.23       0.80       0.58            -1.36             0.91                 0.26                 2.47
ASIAN                                  0.04       1.04       0.17            -0.29             0.36                 0.75                 1.44
BLACK OR AFRICAN AMERICAN              0.27       1.31       0.08             0.12             0.42                 1.13                 1.52
NAT.HAWAIIAN/OTH.PACIFIC              -0.74       0.48       1.00            -2.71             1.22                 0.07                 3.39
WHITE                                 -0.05       0.96       0.06            -0.17             0.08                 0.84                 1.08
HISPANIC OR LATINO OR SPANISH ORIGIN  -0.17       0.84       0.07            -0.30            -0.04                 0.74                 0.96
401.1                                 -0.09       0.91       0.07            -0.23             0.04                 0.80                 1.04
512.7                                 -0.43       0.65       0.07            -0.56            -0.30                 0.57                 0.74
272.1                                 -0.15       0.86       0.07            -0.29            -0.00                 0.75                 1.00
646.0                                  0.01       1.01       0.09            -0.17             0.19                 0.85                 1.21
512.8                                 -0.34       0.71       0.07            -0.48            -0.19                 0.62                 0.82
760.0                                 -0.09       0.91       0.08            -0.24             0.05                 0.78                 1.05
1009.0                                -0.02       0.98       0.08            -0.17             0.13                 0.85                 1.14
530.11                                -0.13       0.88       0.08            -0.28             0.02                 0.76                 1.02
339.0                                 -0.25       0.78       0.09            -0.42            -0.08                 0.66                 0.92
508.0                                  0.09       1.09       0.09            -0.08             0.26                 0.92                 1.29
285.0                                  0.25       1.29       0.08             0.10             0.41                 1.11                 1.50
514.0                                 -0.08       0.92       0.08            -0.24             0.09                 0.78                 1.09
783.0                                  0.23       1.26       0.08             0.07             0.38                 1.07                 1.47
591.0                                 -0.03       0.97       0.09            

In [34]:
coxph_model_data = []
for visit_id in set(cases_250_2[:,0]):
    time_to_diag = min(cases_250_2[:,1][cases_250_2[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_250_2:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(250.2, axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/250_2_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/fitters/coxph_fitter.py:1498: LinAlgWarning: Ill-conditioned matrix (rcond=5.40544e-17): result may not be accurate.
  inv_h_dot_g_T = spsolve(-h, g, assume_a="pos", check_finite=False)
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column NAT.HAWAIIAN/OTH.PACIFIC  have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['phenotype'].astype(bool)
>>> print(df.loc[events, 'NAT.HAWAIIAN/OTH.PACIFIC '].var())
>>> print(df.loc[~events, 'NA

ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [36]:
coxph_model_data = []
for visit_id in set(cases_250_2[:,0]):
    time_to_diag = min(cases_250_2[:,1][cases_250_2[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id][:8] + demographics_data[visit_id][9:] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_250_2:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id][:8] + demographics_data[visit_id][9:] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables[:8] +  demographics_variables[9:] + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(250.2, axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/250_2_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/fitters/coxph_fitter.py:1498: LinAlgWarning: Ill-conditioned matrix (rcond=5.40687e-17): result may not be accurate.
  inv_h_dot_g_T = spsolve(-h, g, assume_a="pos", check_finite=False)
/usr/local/lib/python3.9/site-packages/lifelines/fitters/coxph_fitter.py:1498: LinAlgWarning: Ill-conditioned matrix (rcond=1.69556e-17): result may not be accurate.
  inv_h_dot_g_T = spsolve(-h, g, assume_a="pos", check_finite=False)
/usr/local/lib/python3.9/site-packages/lifelines/fitters/coxph_fitter.py:1498: LinAlgWarning: Ill-conditioned matrix (rcond=6.54549e-17): result may not be accurate.
  inv_h_dot_g_T = spsolve(-h, g, assume_a="pos", check_finite=False)
/usr/local/lib/python3.9/site-packages/lifelines/fitters/coxph_fitter.py:1498: LinAlgWarning: Ill-conditioned matrix (rcond=1.94548e-17): result may not be accurate.
  inv_h_dot_g_T = spsolve(-h, g, assume_a="pos", check_finite=False)
/usr/local/lib/python3.9/site-packages/lifelines/fitters/cox

ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [38]:
coxph_model_data = []
for visit_id in set(cases_250_2[:,0]):
    time_to_diag = min(cases_250_2[:,1][cases_250_2[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id][1:8] + demographics_data[visit_id][9:] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_250_2:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id][1:8] + demographics_data[visit_id][9:] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables[1:8] +  demographics_variables[9:] + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(250.2, axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/250_2_multi_30_cox_new.csv')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 421267 total observations, 420373 right-censored observations>
             duration col = 'days'
                event col = 'phenotype'
      baseline estimation = breslow
   number of observations = 421267
number of events observed = 894
   partial log-likelihood = -9413.64
         time fit was run = 2023-06-16 19:06:07 UTC

---
                                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                    
age_adol                               1.74       5.71       0.71             0.35             3.13                 1.43                22.86
age_adult                              3.19      24.27       0.58             2.05             4.33                 7.77                75.84
age_senior                             3.94      51.52       0.58             2.80             5.08                16.48               161.02
sex_1                                 -0.13       0.88       0.07            -0.27             0.00                 0.76                 1.00
AMERICAN INDIAN OR ALASKA             -0.94       0.39       1.00            -2.91             1.02                 0.05                 2.77
ASIAN                                 -0.17       0.84       0.20            -0.55             0.21                 0.57                 1.24
BLACK OR AFRICAN AMERICAN             -0.09       0.92       0.10            -0.29             0.11                 0.75                 1.11
WHITE                                 -0.54       0.59       0.08            -0.70            -0.38                 0.50                 0.69
HISPANIC OR LATINO OR SPANISH ORIGIN  -0.13       0.88       0.09            -0.30             0.04                 0.74                 1.04
401.1                                 -0.48       0.62       0.10            -0.69            -0.28                 0.50                 0.75
512.7                                 -0.47       0.63       0.11            -0.68            -0.25                 0.51                 0.78
272.1                                 -0.48       0.62       0.13            -0.73            -0.23                 0.48                 0.80
646.0                                 -0.19       0.83       0.19            -0.55             0.18                 0.58                 1.20
512.8                                 -0.36       0.70       0.13            -0.61            -0.10                 0.54                 0.90
760.0                                 -0.35       0.71       0.13            -0.60            -0.09                 0.55                 0.91
1009.0                                -0.32       0.72       0.13            -0.58            -0.06                 0.56                 0.94
530.11                                -0.14       0.87       0.14            -0.40             0.13                 0.67                 1.14
339.0                                 -0.23       0.79       0.15            -0.52             0.05                 0.60                 1.06
508.0                                  0.26       1.29       0.16            -0.06             0.57                 0.95                 1.77
285.0                                 -0.24       0.79       0.15            -0.54             0.06                 0.58                 1.07
514.0                                 -0.66       0.51       0.18            -1.01            -0.31                 0.36                 0.73
783.0                                  0.25       1.28       0.14            -0.02             0.52                 0.98                 1.68
591.0                                  0.14       1.15       0.15            -0.14             0.42                 0.87                 1.53
563.0                                 -0.43       0.65       0.18            -0

In [40]:
coxph_model_data = []
for visit_id in set(cases_282_5[:,0]):
    time_to_diag = min(cases_282_5[:,1][cases_282_5[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_282_5:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/282_5_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1090: ConvergenceWarning: Column(s) [250.2] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column 250.2 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['phenotype

ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [41]:
coxph_model_data = []
for visit_id in set(cases_282_5[:,0]):
    time_to_diag = min(cases_282_5[:,1][cases_282_5[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_282_5:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(250.2, axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/282_5_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column 507.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['phenotype'].astype(bool)
>>> print(df.loc[events, '507.0'].var())
>>> print(df.loc[~events, '507.0'].var())

A very low variance means that the column 507.0 completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), Convergen

ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [42]:
coxph_model_data = []
for visit_id in set(cases_282_5[:,0]):
    time_to_diag = min(cases_282_5[:,1][cases_282_5[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_282_5:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(250.2, axis=1)
coxph_model_data = coxph_model_data.drop(507.0, axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/282_5_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column 508.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['phenotype'].astype(bool)
>>> print(df.loc[events, '508.0'].var())
>>> print(df.loc[~events, '508.0'].var())

A very low variance means that the column 508.0 completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), Convergen

ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [45]:
coxph_model_data = []
for visit_id in set(cases_282_5[:,0]):
    time_to_diag = min(cases_282_5[:,1][cases_282_5[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_282_5:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(250.2, axis=1)
coxph_model_data = coxph_model_data.drop(507.0, axis=1)
coxph_model_data = coxph_model_data.drop(508.0, axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/282_5_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column 716.9 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['phenotype'].astype(bool)
>>> print(df.loc[events, '716.9'].var())
>>> print(df.loc[~events, '716.9'].var())

A very low variance means that the column 716.9 completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), Convergen

ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [46]:
coxph_model_data = []
for visit_id in set(cases_282_5[:,0]):
    time_to_diag = min(cases_282_5[:,1][cases_282_5[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_282_5:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(250.2, axis=1)
coxph_model_data = coxph_model_data.drop(507.0, axis=1)
coxph_model_data = coxph_model_data.drop(508.0, axis=1)
coxph_model_data = coxph_model_data.drop(716.9, axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/282_5_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column AMERICAN INDIAN OR ALASKA have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['phenotype'].astype(bool)
>>> print(df.loc[events, 'AMERICAN INDIAN OR ALASKA'].var())
>>> print(df.loc[~events, 'AMERICAN INDIAN OR ALASKA'].var())

A very low variance means that the column AMERICAN INDIAN OR ALASKA completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-sep

ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [48]:
coxph_model_data = []
for visit_id in set(cases_282_5[:,0]):
    time_to_diag = min(cases_282_5[:,1][cases_282_5[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_282_5:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(250.2, axis=1)
coxph_model_data = coxph_model_data.drop(507.0, axis=1)
coxph_model_data = coxph_model_data.drop(508.0, axis=1)
coxph_model_data = coxph_model_data.drop(716.9, axis=1)
coxph_model_data = coxph_model_data.drop('AMERICAN INDIAN OR ALASKA', axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/282_5_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column ASIAN have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['phenotype'].astype(bool)
>>> print(df.loc[events, 'ASIAN'].var())
>>> print(df.loc[~events, 'ASIAN'].var())

A very low variance means that the column ASIAN completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), Convergen

ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [51]:
coxph_model_data = []
for visit_id in set(cases_282_5[:,0]):
    time_to_diag = min(cases_282_5[:,1][cases_282_5[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_282_5:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(250.2, axis=1)
coxph_model_data = coxph_model_data.drop(507.0, axis=1)
coxph_model_data = coxph_model_data.drop(508.0, axis=1)
coxph_model_data = coxph_model_data.drop(716.9, axis=1)
coxph_model_data = coxph_model_data.drop('AMERICAN INDIAN OR ALASKA', axis=1)
coxph_model_data = coxph_model_data.drop('ASIAN', axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/282_5_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()
/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1110: ConvergenceWarning: Column NAT.HAWAIIAN/OTH.PACIFIC  have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['phenotype'].astype(bool)
>>> print(df.loc[events, 'NAT.HAWAIIAN/OTH.PACIFIC '].var())
>>> print(df.loc[~events, 'NAT.HAWAIIAN/OTH.PACIFIC '].var())

A very low variance means that the column NAT.HAWAIIAN/OTH.PACIFIC  completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-sep

ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [52]:
coxph_model_data = []
for visit_id in set(cases_282_5[:,0]):
    time_to_diag = min(cases_282_5[:,1][cases_282_5[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_282_5:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(250.2, axis=1)
coxph_model_data = coxph_model_data.drop(507.0, axis=1)
coxph_model_data = coxph_model_data.drop(508.0, axis=1)
coxph_model_data = coxph_model_data.drop(716.9, axis=1)
coxph_model_data = coxph_model_data.drop('AMERICAN INDIAN OR ALASKA', axis=1)
coxph_model_data = coxph_model_data.drop('ASIAN', axis=1)
coxph_model_data = coxph_model_data.drop('NAT.HAWAIIAN/OTH.PACIFIC ', axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/282_5_multi_30_cox_new.csv')
cph.print_summary()

/usr/local/lib/python3.9/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()


ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [53]:
coxph_model_data = []
for visit_id in set(cases_282_5[:,0]):
    time_to_diag = min(cases_282_5[:,1][cases_282_5[:,0]==visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] +  previous_conditions_top_30[visit_id] + [covid_prob, 1, time_to_diag]))
    
for visit_id in non_cases_282_5:
    time_to_diag = min([365] + followup_tm_1y[visit_id])
    covid_prob = visit_probability[visit_id]
    coxph_model_data.append((demographics_data[visit_id] + previous_conditions_top_30[visit_id] + [covid_prob, 0, time_to_diag]))
    
coxph_model_data = pd.DataFrame(coxph_model_data, columns=(demographics_variables + top_30 + ['covid_prob','phenotype', 'days']))
coxph_model_data = coxph_model_data.drop(250.2, axis=1)
coxph_model_data = coxph_model_data.drop(507.0, axis=1)
coxph_model_data = coxph_model_data.drop(508.0, axis=1)
coxph_model_data = coxph_model_data.drop(716.9, axis=1)
coxph_model_data = coxph_model_data.drop('AMERICAN INDIAN OR ALASKA', axis=1)
coxph_model_data = coxph_model_data.drop('ASIAN', axis=1)
coxph_model_data = coxph_model_data.drop('NAT.HAWAIIAN/OTH.PACIFIC ', axis=1)
coxph_model_data = coxph_model_data.drop('age_child', axis=1)
cph = CoxPHFitter()
cph.fit(coxph_model_data, 'days', 'phenotype')
cph.summary.to_csv('data/282_5_multi_30_cox_new.csv')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 421272 total observations, 421048 right-censored observations>
             duration col = 'days'
                event col = 'phenotype'
      baseline estimation = breslow
   number of observations = 421272
number of events observed = 224
   partial log-likelihood = -2166.45
         time fit was run = 2023-06-16 19:15:25 UTC

---
                                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                    
age_adol                              -2.37       0.09       0.51            -3.37            -1.37                 0.03                 0.25
age_adult                             -2.03       0.13       0.15            -2.33            -1.73                 0.10                 0.18
age_senior                            -4.32       0.01       0.51            -5.33            -3.32                 0.00                 0.04
sex_1                                 -0.20       0.82       0.14            -0.48             0.07                 0.62                 1.07
BLACK OR AFRICAN AMERICAN              2.63      13.89       0.17             2.29             2.97                 9.90                19.48
WHITE                                 -1.41       0.25       0.34            -2.08            -0.73                 0.12                 0.48
HISPANIC OR LATINO OR SPANISH ORIGIN  -0.23       0.79       0.17            -0.57             0.10                 0.57                 1.11
401.1                                 -3.00       0.05       1.03            -5.01            -0.98                 0.01                 0.37
512.7                                 -0.74       0.48       0.53            -1.77             0.30                 0.17                 1.35
272.1                                 -0.01       0.99       0.76            -1.50             1.48                 0.22                 4.41
646.0                                 -0.04       0.96       0.58            -1.17             1.09                 0.31                 2.96
512.8                                 -1.55       0.21       0.54            -2.62            -0.49                 0.07                 0.61
760.0                                 -0.24       0.79       0.41            -1.04             0.56                 0.35                 1.74
1009.0                                -1.33       0.27       0.47            -2.25            -0.40                 0.11                 0.67
530.11                                -1.93       0.14       1.01            -3.91             0.04                 0.02                 1.04
339.0                                  0.11       1.11       0.37            -0.62             0.84                 0.54                 2.31
285.0                                 -0.83       0.44       0.60            -2.01             0.35                 0.13                 1.42
514.0                                 -1.04       0.35       0.73            -2.46             0.39                 0.09                 1.48
783.0                                 -1.16       0.31       0.42            -1.98            -0.33                 0.14                 0.72
591.0                                  0.14       1.15       0.48            -0.80             1.09                 0.45                 2.96
563.0                                 -1.46       0.23       0.59            -2.62            -0.31                 0.07                 0.73
676.0                                 -0.18       0.83       0.88            -1.90             1.54                 0.15                 4.67
79.0                                  -0.47       0.63       0.42            -1.29             0.35                 0.28                 1.42
386.9                                  0.81       2.24       0.45            -0